In [21]:
import tensorflow as tf

In [22]:
import pandas as pd
from collections import Counter

In [23]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import os
import random
import numpy as np
import pickle

In [24]:
from ReLERNN import *

In [433]:
def SNP_WINDOW(w,r,i,file):
 
    respectiveNormalizedTargets = [normalizeTargets(i,file)]
    targets = np.array(respectiveNormalizedTargets)
    lis=[]

    for i in w:
        co = dict(Counter(i))
        if len(co.keys())==2:
            lis.append(co[min(co,key=co.get)]/len(i))
    bins = np.linspace(0, 1, 20)

    bin_means,bins = np.histogram(lis, bins, normed=True, density=True)
    bins= bins[1:]
    bin_means= bin_means/bin_means.sum()
    

    
    
    print(bin_means.shape)
    return np.array(bins),np.array(targets),np.array(bin_means)



def normalizeTargets(i,file):
    infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+file,"info.p")
    nTargets_1 = pickle.load(open(infoFilename,"rb"))['rho']
    print(len(nTargets_1))
    norm = 'zscore'
    if(norm == 'zscore'):
        tar_mean = np.mean(nTargets_1,axis=0)
        tar_sd = np.std(nTargets_1,axis=0)
        nTargets =nTargets_1- tar_mean
        nTargets = np.divide(nTargets,tar_sd,out=np.zeros_like(nTargets),where=tar_sd!=0)

    return nTargets[i]

In [289]:


def __data_generation1(batchTreeIndices,val):

        haps = []
        pos = []
        for treeIndex in batchTreeIndices:
            print(treeIndex)
            Hfilepath = os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+val+'/val/',str(treeIndex) + "_haps.npy")
            Pfilepath = os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+val+'/val/',str(treeIndex) + "_pos.npy")
            print(Hfilepath)
            H = np.load(Hfilepath)
            P = np.load(Pfilepath)
            haps.append(H)
            pos.append(P)

       
        return haps,pos


            
def __getitem__1(idx,val):
    X,x1= __data_generation1([idx],val)
    return X,x1


In [332]:
os.environ['PYTHONHASHSEED']=str(42)
random.seed(42)
np.random.seed(42)
#nProc = mp.cpu_count()
   

projectDir = '/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool'
trainDir = os.path.join(projectDir,"train")
valiDir = os.path.join(projectDir,"vali")
testDir = os.path.join(projectDir,"test")
networkDir = os.path.join(projectDir,"networks")


test_resultFile = os.path.join(networkDir,"testResults.p")
test_resultFig = os.path.join(networkDir,"testResults.pdf")
modelSave = os.path.join(networkDir,"model.json")
weightsSave = os.path.join(networkDir,"weights.h5")


maxSimS = 0
winFILE=os.path.join(networkDir,"windowSizes.txt")
with open(winFILE, "r") as fIN:
    for line in fIN:
        maxSimS=max([maxSimS, int(line.split()[5])])
maxSegSites = 0

for ds in [trainDir,valiDir,testDir]:
    DsInfoDir = pickle.load(open(os.path.join(ds,"info.p"),"rb"))
    segSitesInDs = max(DsInfoDir["segSites"])
    maxSegSites = max(maxSegSites,segSitesInDs)
maxSegSites = max(maxSegSites, maxSimS)




bds_train_params = {
    'treesDirectory':trainDir,
    'targetNormalization':"zscore",
    'batchSize': 64,
    'maxLen': 448,
    'frameWidth': 5,
    'shuffleInds':True,
    'sortInds':False,
    'center':False,
    'ancVal':-1,
    'padVal':0,
    'derVal':1,
    'realLinePos':True,
    'posPadVal':0,
    'seqD':20,
    'maf':0.05,
    'seed':42
            }


x_1,y = SequenceBatchGenerator(**bds_train_params).__getitem__(0)
x,pos= __getitem__1(0,'train')


vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0],0,'train')

value = tf.convert_to_tensor([vale1],dtype=None)

xTrain = np.reshape(value[0], (value.shape[0], value.shape[1],1))

sites= xTrain.shape

genotype_inputs = layers.Input(shape=(xTrain.shape[1],xTrain.shape[2]))


model1 = layers.Bidirectional(layers.GRU(84,return_sequences=False))(genotype_inputs)
model1 = layers.Dense(256)(model1)
model1 = layers.Dropout(0.35)(model1)
output = layers.Dense(1)(model1)

model1 = Model(inputs=genotype_inputs, outputs=[output])
model1.compile(optimizer='Adam', loss='mse')
model1.summary()





0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/0_haps.npy
13000
(19,)


/tmp/ipykernel_17673/3212070868.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 19, 1)]           0         
                                                                 
 bidirectional_22 (Bidirecti  (None, 168)              43848     
 onal)                                                           
                                                                 
 dense_36 (Dense)            (None, 256)               43264     
                                                                 
 dropout_18 (Dropout)        (None, 256)               0         
                                                                 
 dense_37 (Dense)            (None, 1)                 257       
                                                                 
Total params: 87,369
Trainable params: 87,369
Non-trainable params: 0
______________________________________________________

In [369]:
x_1

array([[[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       ...,

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]], dtype=float32)

array([[0.3646999 , 0.14598169, 0.11495422, 0.06866734, 0.03916582,
        0.06103764, 0.04832146, 0.05595117, 0.06002035, 0.04120041,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [337]:
vali_x=[]
vali_y=[]
for i in range(1000):
    x,pos= __getitem__1(i,'vali')

    
    vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0],i,'vali')
    
    value = tf.convert_to_tensor([vale1],dtype=None)
    xVali = np.reshape(value[0], (value.shape[0], value.shape[1],1))
    

    
 

    vali_x.append(xVali)
    vali_y.append(targets)

0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/0_haps.npy
13000
(19,)
1
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/1_haps.npy
13000
(19,)
2
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/2_haps.npy


/tmp/ipykernel_17673/3212070868.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


13000
(19,)
3
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/3_haps.npy
13000
(19,)
4
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/4_haps.npy
13000
(19,)
5
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/5_haps.npy
13000
(19,)
6
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/6_haps.npy
13000
(19,)
7
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/7_haps.npy
13000
(19,)
8
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/8_haps.npy
13000
(19,)
9
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/9_haps.npy
13000
(19,)
10
/N/u/samishr/Carbonate/Desktop/Recombination_landscape

13000
(19,)
65
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/65_haps.npy
13000
(19,)
66
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/66_haps.npy
13000
(19,)
67
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/67_haps.npy
13000
(19,)
68
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/68_haps.npy
13000
(19,)
69
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/69_haps.npy
13000
(19,)
70
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/70_haps.npy
13000
(19,)
71
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/71_haps.npy
13000
(19,)
72
/N/u/samishr/Carbonate/Desktop/Recombina

13000
(19,)
128
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/128_haps.npy
13000
(19,)
129
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/129_haps.npy
13000
(19,)
130
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/130_haps.npy
13000
(19,)
131
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/131_haps.npy
13000
(19,)
132
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/132_haps.npy
13000
(19,)
133
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/133_haps.npy
13000
(19,)
134
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/134_haps.npy
13000
(19,)
135
/N/u/samishr/Carbonate/De

...

13000
(19,)
745
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/745_haps.npy
13000
(19,)
746
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/746_haps.npy
13000
(19,)
747
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/747_haps.npy
13000
(19,)
748
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/748_haps.npy
13000
(19,)
749
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/749_haps.npy
13000
(19,)
750
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/750_haps.npy
13000
(19,)
751
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/751_haps.npy
13000
(19,)
752
/N/u/samishr/Carbonate/De

13000
(19,)
805
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/805_haps.npy
13000
(19,)
806
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/806_haps.npy
13000
(19,)
807
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/807_haps.npy
13000
(19,)
808
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/808_haps.npy
13000
(19,)
809
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/809_haps.npy
13000
(19,)
810
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/810_haps.npy
13000
(19,)
811
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/811_haps.npy
13000
(19,)
812
/N/u/samishr/Carbonate/De

13000
(19,)
867
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/867_haps.npy
13000
(19,)
868
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/868_haps.npy
13000
(19,)
869
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/869_haps.npy
13000
(19,)
870
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/870_haps.npy
13000
(19,)
871
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/871_haps.npy
13000
(19,)
872
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/872_haps.npy
13000
(19,)
873
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/873_haps.npy
13000
(19,)
874
/N/u/samishr/Carbonate/De

13000
(19,)
929
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/929_haps.npy
13000
(19,)
930
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/930_haps.npy
13000
(19,)
931
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/931_haps.npy
13000
(19,)
932
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/932_haps.npy
13000
(19,)
933
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/933_haps.npy
13000
(19,)
934
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/934_haps.npy
13000
(19,)
935
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/935_haps.npy
13000
(19,)
936
/N/u/samishr/Carbonate/De

13000
(19,)
989
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/989_haps.npy
13000
(19,)
990
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/990_haps.npy
13000
(19,)
991
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/991_haps.npy
13000
(19,)
992
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/992_haps.npy
13000
(19,)
993
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/993_haps.npy
13000
(19,)
994
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/994_haps.npy
13000
(19,)
995
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/995_haps.npy
13000
(19,)
996
/N/u/samishr/Carbonate/De

In [434]:
vali_x=[]
vali_y=[]
val_1_x=[]
val_1_x=[]
for i in range(1000):
    x,pos= __getitem__1(i,'vali')

    
    vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0],i,'vali')
    
    #value = tf.convert_to_tensor([vale1],dtype=None)
    #xVali = np.reshape(value[0], (value.shape[0], value.shape[1],1))
    

    
 

    vali_x.append(vale1)
    vali_y.append(targets)

0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/0_haps.npy
2000
(19,)
1
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/1_haps.npy
2000
(19,)
2
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/2_haps.npy


/tmp/ipykernel_17673/2566818820.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


2000
(19,)
3
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/3_haps.npy
2000
(19,)
4
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/4_haps.npy
2000
(19,)
5
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/5_haps.npy
2000
(19,)
6
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/6_haps.npy
2000
(19,)
7
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/7_haps.npy
2000
(19,)
8
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/8_haps.npy
2000
(19,)
9
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/9_haps.npy
2000
(19,)
10
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN

2000
(19,)
67
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/67_haps.npy
2000
(19,)
68
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/68_haps.npy
2000
(19,)
69
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/69_haps.npy
2000
(19,)
70
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/70_haps.npy
2000
(19,)
71
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/71_haps.npy
2000
(19,)
72
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/72_haps.npy
2000
(19,)
73
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/73_haps.npy
2000
(19,)
74
/N/u/samishr/Carbonate/Desktop/Recombination_lan

2000
(19,)
128
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/128_haps.npy
2000
(19,)
129
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/129_haps.npy
2000
(19,)
130
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/130_haps.npy
2000
(19,)
131
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/131_haps.npy
2000
(19,)
132
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/132_haps.npy
2000
(19,)
133
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/133_haps.npy
2000
(19,)
134
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/134_haps.npy
2000
(19,)
135
/N/u/samishr/Carbonate/Desktop/Re

...

(19,)
696
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/696_haps.npy
2000
(19,)
697
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/697_haps.npy
2000
(19,)
698
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/698_haps.npy
2000
(19,)
699
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/699_haps.npy
2000
(19,)
700
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/700_haps.npy
2000
(19,)
701
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/701_haps.npy
2000
(19,)
702
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/702_haps.npy
2000
(19,)
703
/N/u/samishr/Carbonate/Desktop/Recombi

(19,)
759
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/759_haps.npy
2000
(19,)
760
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/760_haps.npy
2000
(19,)
761
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/761_haps.npy
2000
(19,)
762
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/762_haps.npy
2000
(19,)
763
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/763_haps.npy
2000
(19,)
764
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/764_haps.npy
2000
(19,)
765
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/765_haps.npy
2000
(19,)
766
/N/u/samishr/Carbonate/Desktop/Recombi

2000
(19,)
820
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/820_haps.npy
2000
(19,)
821
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/821_haps.npy
2000
(19,)
822
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/822_haps.npy
2000
(19,)
823
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/823_haps.npy
2000
(19,)
824
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/824_haps.npy
2000
(19,)
825
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/825_haps.npy
2000
(19,)
826
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/826_haps.npy
2000
(19,)
827
/N/u/samishr/Carbonate/Desktop/Re

2000
(19,)
881
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/881_haps.npy
2000
(19,)
882
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/882_haps.npy
2000
(19,)
883
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/883_haps.npy
2000
(19,)
884
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/884_haps.npy
2000
(19,)
885
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/885_haps.npy
2000
(19,)
886
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/886_haps.npy
2000
(19,)
887
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/887_haps.npy
2000
(19,)
888
/N/u/samishr/Carbonate/Desktop/Re

2000
(19,)
944
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/944_haps.npy
2000
(19,)
945
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/945_haps.npy
2000
(19,)
946
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/946_haps.npy
2000
(19,)
947
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/947_haps.npy
2000
(19,)
948
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/948_haps.npy
2000
(19,)
949
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/949_haps.npy
2000
(19,)
950
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/950_haps.npy
2000
(19,)
951
/N/u/samishr/Carbonate/Desktop/Re

In [339]:
callbacks_list = [
    EarlyStopping(
    monitor='val_loss',
    verbose=1,
    min_delta=0.01,
    patience=100),
    ModelCheckpoint(
    filepath='/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val',
    monitor='val_loss',
    save_best_only=True),
    TerminateOnNaN()]




In [435]:

value_vali = tf.convert_to_tensor(vali_x,dtype=None)
tar_vali = tf.convert_to_tensor(vali_y,dtype=None)


In [409]:
lis_=[]
train_=[]
for i in range(13000):
    x,pos= __getitem__1(i,'train')
    vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0],i,'train')
    #value = tf.convert_to_tensor([vale1],dtype=None)

    #xTrain = np.reshape(value[0], (value.shape[0], value.shape[1],1))
    lis_.append(vale1)
    train_.append(targets)
    


0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/0_haps.npy
13000
(19,)
1
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/1_haps.npy
13000
(19,)
2
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/2_haps.npy
13000
(19,)
3
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/3_haps.npy
13000
(19,)
4
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/4_haps.npy
13000
(19,)
5
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/5_haps.npy
13000
(19,)
6
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/6_haps.npy
13000
(19,)
7
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLER

/tmp/ipykernel_17673/3212070868.py:13: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  bin_means,bins = np.histogram(lis, bins, normed=True, density=True)


13000
(19,)
10
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/10_haps.npy
13000
(19,)
11
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/11_haps.npy
13000
(19,)
12
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12_haps.npy
13000
(19,)
13
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/13_haps.npy
13000
(19,)
14
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/14_haps.npy
13000
(19,)
15
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/15_haps.npy
13000
(19,)
16
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/16_haps.npy
13000
(19,)
17
/N/u/samishr/Carbonate/Desktop/Re

(19,)
72
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/72_haps.npy
13000
(19,)
73
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/73_haps.npy
13000
(19,)
74
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/74_haps.npy
13000
(19,)
75
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/75_haps.npy
13000
(19,)
76
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/76_haps.npy
13000
(19,)
77
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/77_haps.npy
13000
(19,)
78
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/78_haps.npy
13000
(19,)
79
/N/u/samishr/Carbonate/Desktop/Recombin

(19,)
135
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/135_haps.npy
13000
(19,)
136
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/136_haps.npy
13000
(19,)
137
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/137_haps.npy
13000
(19,)
138
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/138_haps.npy
13000
(19,)
139
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/139_haps.npy
13000
(19,)
140
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/140_haps.npy
13000
(19,)
141
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/141_haps.npy
13000
(19,)
142
/N/u/samishr/Carbonate/D

...

13000
(19,)
12723
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12723_haps.npy
13000
(19,)
12724
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12724_haps.npy
13000
(19,)
12725
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12725_haps.npy
13000
(19,)
12726
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12726_haps.npy
13000
(19,)
12727
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12727_haps.npy
13000
(19,)
12728
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12728_haps.npy
13000
(19,)
12729
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12729_haps.npy
13000


13000
(19,)
12783
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12783_haps.npy
13000
(19,)
12784
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12784_haps.npy
13000
(19,)
12785
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12785_haps.npy
13000
(19,)
12786
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12786_haps.npy
13000
(19,)
12787
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12787_haps.npy
13000
(19,)
12788
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12788_haps.npy
13000
(19,)
12789
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12789_haps.npy
13000


(19,)
12841
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12841_haps.npy
13000
(19,)
12842
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12842_haps.npy
13000
(19,)
12843
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12843_haps.npy
13000
(19,)
12844
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12844_haps.npy
13000
(19,)
12845
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12845_haps.npy
13000
(19,)
12846
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12846_haps.npy
13000
(19,)
12847
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12847_haps.npy
13000
(19,)


13000
(19,)
12900
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12900_haps.npy
13000
(19,)
12901
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12901_haps.npy
13000
(19,)
12902
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12902_haps.npy
13000
(19,)
12903
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12903_haps.npy
13000
(19,)
12904
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12904_haps.npy
13000
(19,)
12905
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12905_haps.npy
13000
(19,)
12906
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12906_haps.npy
13000


(19,)
12959
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12959_haps.npy
13000
(19,)
12960
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12960_haps.npy
13000
(19,)
12961
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12961_haps.npy
13000
(19,)
12962
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12962_haps.npy
13000
(19,)
12963
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12963_haps.npy
13000
(19,)
12964
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12964_haps.npy
13000
(19,)
12965
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/12965_haps.npy
13000
(19,)


In [410]:
value_1 = tf.convert_to_tensor(lis_,dtype=None)
tar = tf.convert_to_tensor(train_,dtype=None)

In [437]:
history = model1.fit(value_1,tar,
steps_per_epoch=1000,
epochs=15,
callbacks=callbacks_list,
validation_data =(value_vali,tar_vali),
use_multiprocessing=True)
#model1.fit(xTrain,targets,epochs=5)
    
    
model_json = model1.to_json()
with open('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/model', "w+") as json_file:
    json_file.write(model_json)


history.history['loss'] = np.array(history.history['loss'])
#history.history['val_loss'] = np.array(history.history['val_loss'])
history.history['predictions'] = np.array(predictions)
history.history['Y_test'] = np.array(targets)
history.history['name'] = 'ModelName'

pickle.dump(history.history, open( 'resultsFile', "wb" ))

Epoch 1/15
1000/1000 [==============================] - 13s 13ms/step - loss: 0.7832 - val_loss: 0.6830
Epoch 2/15
1000/1000 [==============================] - 13s 13ms/step - loss: 0.6948 - val_loss: 0.6194
Epoch 3/15
1000/1000 [==============================] - 13s 13ms/step - loss: 0.6235 - val_loss: 0.5475
Epoch 4/15
1000/1000 [==============================] - 13s 13ms/step - loss: 0.5167 - val_loss: 0.4788
Epoch 5/15
1000/1000 [==============================] - 13s 13ms/step - loss: 0.4743 - val_loss: 0.4967
Epoch 6/15
1000/1000 [==============================] - 13s 13ms/step - loss: 0.4540 - val_loss: 0.4448
Epoch 7/15
1000/1000 [==============================] - 13s 13ms/step - loss: 0.4367 - val_loss: 0.4251
Epoch 8/15
1000/1000 [==============================] - 13s 13ms/step - loss: 0.4334 - val_loss: 0.3926
Epoch 9/15
1000/1000 [==============================] - 13s 13ms/step - loss: 0.4270 - val_loss: 0.3870
Epoch 10/15
1000/1000 [==============================] - 13s 13m

In [438]:
infoFilename= os.path.join("resultsFile")
ne = pickle.load(open(infoFilename,"rb"))
ne

{'loss': array([0.78324431, 0.69479352, 0.62352622, 0.51669371, 0.47427309,
        0.45402354, 0.43671808, 0.43342999, 0.42696154, 0.42356008,
        0.42186344, 0.41672888, 0.41232663, 0.40813702, 0.40872738]),
 'val_loss': [0.6829861402511597,
  0.6194019913673401,
  0.5475037097930908,
  0.4788304567337036,
  0.49671807885169983,
  0.44479912519454956,
  0.4251220226287842,
  0.39256346225738525,
  0.3869553506374359,
  0.38743266463279724,
  0.3909316658973694,
  0.3891775906085968,
  0.41683733463287354,
  0.389824241399765,
  0.38885390758514404],
 'predictions': array([[-0.01817163]], dtype=float32),
 'Y_test': array([-1.35504269]),
 'name': 'ModelName'}

In [439]:
model1.predict(value_vali)

array([[-9.13086653e-01],
       [-1.30592659e-01],
       [-2.11918578e-01],
       [ 1.28215700e-01],
       [ 3.11954141e-01],
       [ 3.39920148e-02],
       [-1.46203053e+00],
       [ 4.91797447e-01],
       [-1.49912643e+00],
       [-1.16382170e+00],
       [ 1.41345441e-01],
       [ 1.23065010e-01],
       [-6.93754077e-01],
       [ 4.51641619e-01],
       [ 6.52425110e-01],
       [-1.01874673e+00],
       [-2.01148689e-02],
       [ 5.11363804e-01],
       [ 8.84747207e-01],
       [-1.06873763e+00],
       [ 8.12178493e-01],
       [ 7.71541655e-01],
       [-1.45203495e+00],
       [-1.27922153e+00],
       [-9.82371926e-01],
       [-4.85315382e-01],
       [ 8.47683370e-01],
       [ 3.61143470e-01],
       [-4.39169228e-01],
       [ 3.51146817e-01],
       [ 1.35286838e-01],
       [ 6.41624510e-01],
       [ 1.60978273e-01],
       [-1.28880608e+00],
       [ 7.00487077e-01],
       [ 1.14454031e-01],
       [-7.89381802e-01],
       [-1.13585436e+00],
       [ 3.6

In [415]:
tar_vali

<tf.Tensor: shape=(1000, 1), dtype=float64, numpy=
array([[-4.21957787e-01],
       [ 1.57722750e+00],
       [ 8.18320655e-01],
       [ 3.55678817e-01],
       [-1.18017454e+00],
       [-1.18025824e+00],
       [-1.51998544e+00],
       [ 1.28390050e+00],
       [ 3.64202374e-01],
       [ 7.35319298e-01],
       [-1.65009832e+00],
       [ 1.64383141e+00],
       [ 1.16685338e+00],
       [-9.84756114e-01],
       [-1.09063282e+00],
       [-1.08515219e+00],
       [-6.65874439e-01],
       [ 9.92565416e-02],
       [-2.22776648e-01],
       [-7.11026763e-01],
       [ 4.01460235e-01],
       [-1.23751154e+00],
       [-7.07850171e-01],
       [-4.50334428e-01],
       [-1.39068843e-01],
       [ 1.00284942e+00],
       [-1.02870175e+00],
       [ 6.27477662e-02],
       [ 3.34013939e-01],
       [-1.56034983e+00],
       [ 3.86512367e-01],
       [-1.12984402e+00],
       [-1.49580823e+00],
       [ 1.57088211e+00],
       [ 1.62898841e+00],
       [ 1.08342202e+00],
       [-6.64

In [440]:
erro=model1.predict(value_vali)-tar_vali

In [446]:
de1= pd.DataFrame(erro)

In [442]:
def z_normalizeTargets(error):
    infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/',"info.p")
    nTargets_1 = pickle.load(open(infoFilename,"rb"))['rho']

    tar_mean = np.mean(nTargets_1,axis=0)
    tar_sd = np.std(nTargets_1,axis=0)
    erro = np.dot(error,tar_sd)+tar_mean


    return erro

In [448]:
erro= z_normalizeTargets(erro)

In [449]:
de1= pd.DataFrame(erro)

In [450]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.clf()
sns.boxplot(data=de1)

plt.savefig("/N/u/samishr/Carbonate/Desktop/Recombination_landscape/Data_Processing/fig1.png")

In [190]:
infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/',"info.p")
ne = pickle.load(open(infoFilename,"rb"))['Ne']

In [51]:
nTargets_1

69982.4499438376

In [55]:
mu = pickle.load(open(infoFilename,"rb"))['mu']

In [53]:
nTargets_1

{'N': 20,
 'Ne': 69982.4499438376,
 'priorLowsRho': 0.0,
 'priorHighsRho': 1e-08,
 'priorLowsMu': 6.6e-09,
 'priorHighsMu': 1.3300000000000002e-08,
 'ChromosomeLength': 136200,
 'MspDemographics': None,
 'rho': array([3.74540119e-09, 9.50714306e-09, 7.31993942e-09, ...,
        8.67618866e-09, 3.66456646e-09, 4.32136995e-10]),
 'hotWin': None,
 'mu': array([1.31279282e-08, 6.71505904e-09, 1.29492661e-08, ...,
        1.12837416e-08, 1.16580924e-08, 1.16752855e-08]),
 'segSites': array([1966,  909, 1761, ..., 1459, 1690, 1989]),
 'winMasks': [[0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0513950073421439, [[0, 7000]], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.06607929515418502, [[0, 9000]], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.25697503671071953, [[0, 35000]], 0],
  [0.0, [], 0],
  [0.02936857562408223, [[0